# Import packages and set up

In [1]:
import ee
import geemap

In [2]:
ee.Authenticate()
ee.Initialize(project='ee-curuai')

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Import Chips dataset

Climate Hazards Center InfraRed Precipitation with Station data (CHIRPS) is a 30+ year quasi-global rainfall dataset. CHIRPS incorporates 0.05° resolution satellite imagery with in-situ station data to create gridded rainfall time series for trend analysis and seasonal drought monitoring.

Resolution
5566 meters

In [42]:
chirps = ee.ImageCollection("UCSB-CHG/CHIRPS/DAILY")


import region of interest

In [6]:
floddplain = ee.FeatureCollection('projects/ee-curuai/assets/vectors/varzea_alagavel')

# Accumulate period precipitaion and obtain the mean in the floddplain

In [7]:
# //Definir o intervalo de tempo
startyear = 2000
endyear = 2024

In [28]:
# //Criar uma lista sequencial para ano
years = ee.List.sequence(startyear,endyear);

# //Criar uma lista sequencial para meses
months = ee.List([1,4,7,10])

# //Definir nossa data inicial e final
startdate = ee.Date.fromYMD(startyear,1,1);
enddate = ee.Date.fromYMD(endyear,12,31);

In [9]:
# //filterando a coleção
chirps = chirps.select('precipitation').filterDate(startdate, enddate).filterBounds(floddplain)
print('qtds',chirps.size().getInfo())

qtds 1788


In [10]:
scale = chirps.first().projection().nominalScale()

In [43]:
# //Calculo da precipitação de cada periodo da série

precipitacao_acum =  ee.ImageCollection.fromImages(years.map(lambda y : months.map( lambda m : chirps.filter(ee.Filter.calendarRange(y,y,'year')).filter(ee.Filter.calendarRange(m,ee.Number(m).add(2),'month')).sum().set({'year': y, 'month': m,'system:time_start': ee.Date.fromYMD(y, m, 1)}))).flatten() )

display(precipitacao_acum.limit(10))

In [44]:
precipitacao_acum.size()

In [45]:
# //Calculo da precipitação de cada periodo da série - Opção de média diária.... posso fazer outras médias
# precipitacao_med =  ee.ImageCollection.fromImages(years.map(lambda y : months.map( lambda m : chirps.filter(ee.Filter.calendarRange(y,y,'year')).filter(ee.Filter.calendarRange(m,ee.Number(m).add(2),'month')).mean().set({'year': y, 'month': m,'system:time_start': ee.Date.fromYMD(y, m, 1)}))).flatten() )

# display(precipitacao_med.limit(10))

# Obtain mean period accumulated precipitation value in the floodplain.

In [50]:
def mean_precip(img):
  '''receives a precipitaion image returns mean value within the floodplain limits
  as a property of the input image'''


  floodplain = ee.FeatureCollection('projects/ee-curuai/assets/vectors/varzea_alagavel')


  stats = img.select("precipitation").reduceRegion(**{
    'reducer': ee.Reducer.mean(),#.combine(
    # reducer2=ee.Reducer.stdDev(), sharedInputs=True).combine(
    # reducer2=ee.Reducer.max(), sharedInputs=True).combine(
    # reducer2=ee.Reducer.min(), sharedInputs=True),
    'geometry': floodplain.geometry(),
    'scale': chirps.first().projection().nominalScale(),
    'maxPixels': 1e10
    })

  return img.clip(floodplain).set({'mean_precipitation':stats.get('precipitation'),
                  # 'max_SPM':stats.get('classification_max'),
                  # 'min_SPM':stats.get('classification_min'),
                  # 'stdDev_SPM':stats.get('classification_stdDev')
                  })

In [51]:
precip_acum = precipitacao_acum.map(mean_precip)
precip_acum.limit(5)


# exportar e adicionar a pasta de datasets

# fazeer memso processo com dados de vento nas duas direções
# dados de vento
# ee.ImageCollection("ECMWF/ERA5/MONTHLY")